In [13]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [2]:
# Load dataset from the cloud URL
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
df = pd.read_csv(url)

# Drop non-beneficial columns (EIN and NAME)
df = df.drop(columns=["EIN", "NAME"])

# Group less common APPLICATION_TYPEs into "Other"
app_cutoff = 500  # Adjust as needed
app_to_replace = df["APPLICATION_TYPE"].value_counts()[df["APPLICATION_TYPE"].value_counts() < app_cutoff].index
df["APPLICATION_TYPE"] = df["APPLICATION_TYPE"].replace(app_to_replace, "Other")

# Group less common CLASSIFICATIONs into "Other"
class_cutoff = 1000  # Adjust as needed
class_to_replace = df["CLASSIFICATION"].value_counts()[df["CLASSIFICATION"].value_counts() < class_cutoff].index
df["CLASSIFICATION"] = df["CLASSIFICATION"].replace(class_to_replace, "Other")

# One-hot encode categorical variables
df = pd.get_dummies(df, dtype=int)

# Split into features (X) and target (y)
X = df.drop(columns=["IS_SUCCESSFUL"]).values
y = df["IS_SUCCESSFUL"].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Check correlation of numerical features with the target variable
correlation = df.corr()["IS_SUCCESSFUL"].sort_values(ascending=False)
print(correlation)


IS_SUCCESSFUL                   1.000000
AFFILIATION_Independent         0.368322
ORGANIZATION_Trust              0.162164
CLASSIFICATION_Other            0.111202
APPLICATION_TYPE_T5             0.088575
APPLICATION_TYPE_T10            0.086805
APPLICATION_TYPE_T6             0.079817
INCOME_AMT_1-9999               0.067058
CLASSIFICATION_C1000            0.062589
ORGANIZATION_Co-operative       0.053520
APPLICATION_TYPE_Other          0.045165
INCOME_AMT_10000-24999          0.036937
INCOME_AMT_100000-499999        0.030542
INCOME_AMT_25000-99999          0.026239
USE_CASE_Preservation           0.024145
ORGANIZATION_Corporation        0.016690
AFFILIATION_Other               0.010121
APPLICATION_TYPE_T7             0.009752
USE_CASE_Other                  0.008765
SPECIAL_CONSIDERATIONS_Y        0.005469
AFFILIATION_National            0.004581
AFFILIATION_Regional            0.003238
AFFILIATION_Family/Parent       0.002608
CLASSIFICATION_C2000           -0.000434
APPLICATION_TYPE

In [21]:
# Drop low-correlation and noisy features
df = df.drop(columns=[
    "ASK_AMT", "SPECIAL_CONSIDERATIONS_N", "SPECIAL_CONSIDERATIONS_Y",
    "STATUS", "USE_CASE_Other", "AFFILIATION_Family/Parent",
    "AFFILIATION_Regional", "APPLICATION_TYPE_T3", "APPLICATION_TYPE_T7"
])

# Re-define X (features) and y (target)
X = df.drop(columns=["IS_SUCCESSFUL"]).values  # Features
y = df["IS_SUCCESSFUL"].values  # Target

In [ ]:
# Initialize the Sequential model
nn = Sequential()

# Explicitly define input shape
nn.add(Input(shape=(X_train_scaled.shape[1],)))

# First hidden layer (more neurons)
nn.add(Dense(units=128, activation="relu"))  #First attempted with 100 units, then finally with 128 units

# Second hidden layer (more neurons)
nn.add(Dense(units=64, activation="relu"))   #First attempted with 50 units, then finally with 64 units

# Third hidden layer (NEW LAYER)
nn.add(Dense(units=32, activation="relu"))  # Added third hidden layer of 25 units after the first attempt. Increased to 32 units in the third attempt

# Output layer (Binary classification)
nn.add(Dense(units=1, activation="sigmoid"))

# Check the model structure
nn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                     │ (None, 128)                 │           5,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,001 (62.50 KB)

 Trainable params: 16,001 (62.50 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the optimized model
history = nn.fit(
    X_train_scaled, y_train,
    epochs=250,  # Increased epochs to 150 on first attemt, 200 on second attempt, then to 250 on the third attempt
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    verbose=1
)


Epoch 1/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7152 - loss: 0.5816 - val_accuracy: 0.7262 - val_loss: 0.5660
Epoch 2/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7296 - loss: 0.5563 - val_accuracy: 0.7255 - val_loss: 0.5652
Epoch 3/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7290 - loss: 0.5521 - val_accuracy: 0.7278 - val_loss: 0.5612
Epoch 4/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7343 - loss: 0.5463 - val_accuracy: 0.7270 - val_loss: 0.5604
Epoch 5/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7330 - loss: 0.5482 - val_accuracy: 0.7223 - val_loss: 0.5568
Epoch 6/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7343 - loss: 0.5450 - val_accuracy: 0.7273 - val_loss: 0.5581
Epoch 7/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7308 - loss: 0.5460 - val_accuracy: 0.7251 - val_loss: 0.5592
Epoch 8/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7362 - loss: 0.5416 - val_accu

In [25]:
# Evaluate model performance
loss, accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7210 - loss: 0.6110
Test Loss: 0.6136, Test Accuracy: 0.7230
